In [1]:
# imports
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 


database = "pythonsqlite.db"
 
# create a database connection
conn = create_connection(database)

2.6.0


# /* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 1 of the case study, which means that there'll be more guidance for you about how to 
setup your local SQLite connection in PART 2 of the case study. 

The questions in the case study are exactly the same as with Tier 2. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */




# QUESTIONS 1-9 were first run on PHPMyAdmin. When they were transferred to SQLITE, the CONCAT function was repl

# /* Q1: 
Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

In [3]:
# query and print
with conn:
    cur = conn.cursor()
    
    query01 = """
        SELECT name
          FROM Facilities
        WHERE membercost > 0
        """
    rs01=cur.execute(query01)
 
    df01=pd.DataFrame(rs01.fetchall())
    df01.columns = [i[0] for i in rs01.description]
    print(df01)

             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court


# /* Q2: 
How many facilities do not charge a fee to members? */

In [4]:
# query and print
with conn:
    cur = conn.cursor()
    
    query02 = """
        SELECT COUNT(*) AS count_no_fee
          FROM Facilities
         WHERE membercost = 0
        """
    rs02=cur.execute(query02)
 
    df02=pd.DataFrame(rs02.fetchall())
    df02.columns = [i[0] for i in rs02.description]
    print(df02)

   count_no_fee
0             4


# /* Q3:
Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

In [5]:
# query and print
with conn:
    cur = conn.cursor()
    
    query03 = """
        SELECT facid
              ,name
              ,membercost
              ,monthlymaintenance
          FROM Facilities
         WHERE membercost > 0
           AND membercost < (.2*monthlymaintenance)
        """
    rs03=cur.execute(query03)
    df03=pd.DataFrame(rs03.fetchall())
    df03.columns = [i[0] for i in rs03.description]
    print(df03)


   facid            name  membercost  monthlymaintenance
0      0  Tennis Court 1         5.0                 200
1      1  Tennis Court 2         5.0                 200
2      4  Massage Room 1         9.9                3000
3      5  Massage Room 2         9.9                3000
4      6    Squash Court         3.5                  80


# /* Q4:
Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

In [6]:
# query and print
with conn:
    cur = conn.cursor()
    
    query04 = """
        SELECT *
          FROM Facilities
         WHERE facid in (1,5)
        """
    rs04=cur.execute(query04)
    df04=pd.DataFrame(rs04.fetchall())
    df04.columns = [i[0] for i in rs04.description]
    print(df04)


   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


# /* Q5:
Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

In [7]:
# query and print
with conn:
    cur = conn.cursor()
    
    query05 = """
        SELECT name
              ,monthlymaintenance
              ,CASE WHEN monthlymaintenance > 100 THEN 'Expensive'
                    ELSE 'Cheap' END AS Category
          FROM Facilities
        """
    rs05=cur.execute(query05)
    df05=pd.DataFrame(rs05.fetchall())
    df05.columns = [i[0] for i in rs05.description]
    print(df05)


              name  monthlymaintenance   Category
0   Tennis Court 1                 200  Expensive
1   Tennis Court 2                 200  Expensive
2  Badminton Court                  50      Cheap
3     Table Tennis                  10      Cheap
4   Massage Room 1                3000  Expensive
5   Massage Room 2                3000  Expensive
6     Squash Court                  80      Cheap
7    Snooker Table                  15      Cheap
8       Pool Table                  15      Cheap


# /* Q6:
You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

In [8]:
# query and print
with conn:
    cur = conn.cursor()
    
    query06 = """
        SELECT m.firstname
              ,m.surname
              ,m.joindate
          FROM Members as m
         WHERE joindate = (SELECT MAX(joindate) FROM Members)
        """
    rs06=cur.execute(query06)
    df06=pd.DataFrame(rs06.fetchall())
    df06.columns = [i[0] for i in rs06.description]
    print(df06)


  firstname surname             joindate
0    Darren   Smith  2012-09-26 18:08:45


# /* Q7:
Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

In [9]:
# query and print
with conn:
    cur = conn.cursor()
    
    query07 = """
        SELECT DISTINCT
               (m.firstname || ' ' || m.surname) AS member
              ,f.name
          FROM Bookings AS b
                 INNER JOIN
               Facilities as f
                   ON b.facid = f.facid
                 INNER JOIN
               Members as m
                   ON b.memid = m.memid
         WHERE f.name LIKE 'Tennis%'          
         ORDER BY member
         
        """
   
    rs07=cur.execute(query07)
    df07=pd.DataFrame(rs07.fetchall())
    df07.columns = [i[0] for i in rs07.description]
    print(df07)


               member            name
0          Anne Baker  Tennis Court 1
1          Anne Baker  Tennis Court 2
2        Burton Tracy  Tennis Court 2
3        Burton Tracy  Tennis Court 1
4        Charles Owen  Tennis Court 1
5        Charles Owen  Tennis Court 2
6        Darren Smith  Tennis Court 2
7       David Farrell  Tennis Court 1
8       David Farrell  Tennis Court 2
9         David Jones  Tennis Court 2
10        David Jones  Tennis Court 1
11       David Pinker  Tennis Court 1
12      Douglas Jones  Tennis Court 1
13      Erica Crumpet  Tennis Court 1
14     Florence Bader  Tennis Court 2
15     Florence Bader  Tennis Court 1
16        GUEST GUEST  Tennis Court 2
17        GUEST GUEST  Tennis Court 1
18     Gerald Butters  Tennis Court 1
19     Gerald Butters  Tennis Court 2
20   Henrietta Rumney  Tennis Court 2
21         Jack Smith  Tennis Court 1
22         Jack Smith  Tennis Court 2
23    Janice Joplette  Tennis Court 1
24    Janice Joplette  Tennis Court 2
25     Jemim

# /* Q8:
Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

In [10]:
# query and print
with conn:
    cur = conn.cursor()
    
    query08 = """
        SELECT f.name
              ,(m.firstname || ' '|| m.surname) AS member

              ,case when b.memid = 0 then slots * guestcost
                    else slots * membercost END AS cost
          from Bookings as b
                 inner join
               Facilities as f
                   on b.facid = f.facid
                 inner join
               Members as m
                   on b.memid = m.memid
         WHERE DATE(starttime) = '2012-09-14'
           AND (case when b.memid = 0 then slots * guestcost
                     else slots * membercost END) > 30
         ORDER BY cost desc
        """
    rs08=cur.execute(query08)
    df08=pd.DataFrame(rs08.fetchall())
    df08.columns = [i[0] for i in rs08.description]
    print(df08)


              name          member   cost
0   Massage Room 2     GUEST GUEST  320.0
1   Massage Room 1     GUEST GUEST  160.0
2   Massage Room 1     GUEST GUEST  160.0
3   Massage Room 1     GUEST GUEST  160.0
4   Tennis Court 2     GUEST GUEST  150.0
5   Tennis Court 1     GUEST GUEST   75.0
6   Tennis Court 1     GUEST GUEST   75.0
7   Tennis Court 2     GUEST GUEST   75.0
8     Squash Court     GUEST GUEST   70.0
9   Massage Room 1  Jemima Farrell   39.6
10    Squash Court     GUEST GUEST   35.0
11    Squash Court     GUEST GUEST   35.0


# /* Q9:
This time, produce the same result as in Q8, but using a subquery. */

In [11]:
# query and print
with conn:
    cur = conn.cursor()
    
    query09 = """
        SELECT costs.name
              ,(m.firstname || ' ' || m.surname) AS member
              ,costs.cost
          from (select b.memid
                      ,f.name
	                  ,case when b.memid = 0 then slots * guestcost
                            else slots * membercost END AS cost
                  from Bookings as b
                         inner join
                       Facilities as f
                           on b.facid = f.facid
		         WHERE DATE(starttime) = '2012-09-14'
	          ) as costs
                inner join
              Members as m
                  on costs.memid = m.memid
         WHERE costs.cost > 30
         ORDER BY costs.cost desc
;
        """
    rs09=cur.execute(query09)
    df09=pd.DataFrame(rs09.fetchall())
    df09.columns = [i[0] for i in rs09.description]
    print(df09)


              name          member   cost
0   Massage Room 2     GUEST GUEST  320.0
1   Massage Room 1     GUEST GUEST  160.0
2   Massage Room 1     GUEST GUEST  160.0
3   Massage Room 1     GUEST GUEST  160.0
4   Tennis Court 2     GUEST GUEST  150.0
5   Tennis Court 1     GUEST GUEST   75.0
6   Tennis Court 1     GUEST GUEST   75.0
7   Tennis Court 2     GUEST GUEST   75.0
8     Squash Court     GUEST GUEST   70.0
9   Massage Room 1  Jemima Farrell   39.6
10    Squash Court     GUEST GUEST   35.0
11    Squash Court     GUEST GUEST   35.0


# /* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.


In [12]:
# query and print
with conn:
    cur = conn.cursor()
    
    query10 = """
        SELECT f.name
	          ,sum(case when b.memid = 0 then slots * guestcost
                        else slots * membercost END) AS revenue
          FROM Bookings as b
                 INNER JOIN
               Facilities as f
	               on b.facid = f.facid
         GROUP BY f.name
        HAVING revenue < 1000
         ORDER BY revenue
        """
    rs10=cur.execute(query10)
    df10=pd.DataFrame(rs10.fetchall())
    df10.columns = [i[0] for i in rs10.description]
    print(df10)


            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


# /* Q11:
Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [13]:
# query and print
with conn:
    cur = conn.cursor()
    
    query11 = """
        SELECT (m.surname || ', ' || m.firstname) AS member
              ,CASE WHEN m.recommendedby = 0 THEN 'no recommendation'
	                ELSE (r.surname || ', ' || r.firstname)
			        END as recommended_by
          FROM Members as m
                 LEFT JOIN
	           Members as r
	               on m.recommendedby = r.memid
          WHERE m.memid <> 0
          ORDER BY member
        """
    rs11=cur.execute(query11)
    df11=pd.DataFrame(rs11.fetchall())
    df11.columns = [i[0] for i in rs11.description]
    print(df11)


                      member      recommended_by
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8             Farrell, David                None
9            Farrell, Jemima                None
10          Genting, Matthew     Butters, Gerald
11                Hunt, John  Purview, Millicent
12              Jones, David    Joplette, Janice
13            Jones, Douglas        Jones, David
14          Joplette, Janice       Smith, Darren
15           Mackenzie, Anna       Smith, Darren
16             Owen, Charles       Smith, Darren
17             Pinker, David     Farrell, Jemima
18        Purview, Millicent        Smith, Tracy
19               Row

# /* Q12:
Find the facilities with their usage by member, but not guests */

In [14]:
# query and print
with conn:
    cur = conn.cursor()
    
    query12 = """
        SELECT f.name
              ,count(b.bookid) AS member_bookings
          FROM Facilities as f
	             INNER JOIN
	           Bookings as b
	               on b.facid = f.facid 
         WHERE b.memid <> 0
         GROUP BY f.name
        """
    rs12=cur.execute(query12)
    df12=pd.DataFrame(rs12.fetchall())
    df12.columns = [i[0] for i in rs12.description]
    print(df12)


              name  member_bookings
0  Badminton Court              344
1   Massage Room 1              421
2   Massage Room 2               27
3       Pool Table              783
4    Snooker Table              421
5     Squash Court              195
6     Table Tennis              385
7   Tennis Court 1              308
8   Tennis Court 2              276


# /* Q13:
Find the facilities usage by month, but not guests */

In [15]:
# query and print
with conn:
    cur = conn.cursor()
    
    query13 = """
        SELECT f.name
              ,strftime('%m', b.starttime) AS month
              ,count(b.bookid) AS member_bookings
          FROM Facilities as f
                 INNER JOIN
               Bookings as b
                   on b.facid = f.facid 
         WHERE b.memid <> 0
         GROUP BY month,f.name
         ORDER BY f.name,month
        """
    rs13=cur.execute(query13)
    df13=pd.DataFrame(rs13.fetchall())
    df13.columns = [i[0] for i in rs13.description]
    print(df13)


               name month  member_bookings
0   Badminton Court    07               51
1   Badminton Court    08              132
2   Badminton Court    09              161
3    Massage Room 1    07               77
4    Massage Room 1    08              153
5    Massage Room 1    09              191
6    Massage Room 2    07                4
7    Massage Room 2    08                9
8    Massage Room 2    09               14
9        Pool Table    07              103
10       Pool Table    08              272
11       Pool Table    09              408
12    Snooker Table    07               68
13    Snooker Table    08              154
14    Snooker Table    09              199
15     Squash Court    07               23
16     Squash Court    08               85
17     Squash Court    09               87
18     Table Tennis    07               48
19     Table Tennis    08              143
20     Table Tennis    09              194
21   Tennis Court 1    07               65
22   Tennis